# DATA CLEANING

This notebook contains majority of the code that we used to clean and fill the NaN values in our dataset. You can find more detailed explanations of the columns we manipulated below.

In [ ]:
!pip install contextily
import contextily as ctx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 24.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Point
import matplotlib.cm as cm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/Project 3 (Arrest Dataset)/Arrest_Data_from_2020_to_Present.csv')

# **Data Cleaning**

This text cell explains how we dealt with each of the columns that had missing values.

## Cross Street

- Given that this value is missing in over half of the data, we decided to drop the column. We already have plenty of data regarding location anyways.

## Booking Date

- First filled missing Booking Dates with the Arrest Date
- Then analyzed time differences to identify overnight bookings (found 16.1% of bookings cross to the next day)
- Automatically adjusted 6,576 booking dates for overnight bookings
- Fixed 24 inconsistencies where booking time was earlier than arrest time but dates were the same

## Booking Time

- Calculated the median time difference between arrest and booking (157 minutes)
- Successfully filled all 75,378 missing booking times using this pattern
- Properly handled cases where bookings cross into the next day
- Converted properly to military time format stored as floats

## Booking Location

- Used a 2 step approach to fill missing Booking Locations:
  1. First filled 72,932 locations using Address → Booking Location mapping
  2. Filled additional 2,487 locations using Area Name → Booking Location mapping

## Booking Location Code

- Created mapping from Booking Location → Booking Location Code
- Successfully filled all 75,382 missing Booking Location Codes
- Achieved 100% completion for this column

## Arrest Type Code

- Dropped 22 rows with missing Arrest Type Code values

## Time

- Dropped rows with missing Time values (part of the 22 rows above)
- These are critical for booking time calculations

## Charge Description, Disposition Description, Charge Group Code, Charge Group Description

- Used a multi-step approach that significantly reduced missing values:
  1. Mapped from 274 frequent charges (≥100 occurrences) to fill 5,709 charge descriptions and codes
  2. Used charge code prefixes to fill additional 17,153 values
  3. Leveraged relationships between columns to fill more missing values
  4. After all strategies, reduced missing values from ~32,000 to only ~9,300
- Finally dropped the remaining rows with missing values (~2.8% of dataset), we chose to do this because filling out the rest of the missing values with things like the most common values would just be incorrect. Our approach fills out all the possible values with real connections between the columns.
- This approach preserved data integrity while ensuring a complete dataset


# NOTE

- The following cell takes about ~3 minutes to run
- The results of this code and cleaning can be used via the `clean_arrest_data.csv` file inside the `data` folder.

In [ ]:
def fix_booking_date_and_time(df):
    """
    Fix booking date and time together, properly handling day changes

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with Time, Booking Time, Arrest Date, and Booking Date columns

    Returns:
    --------
    pandas.DataFrame
        DataFrame with fixed Booking Date and Time
    """
    clean_df = df.copy()

    if ('Booking Time' in clean_df.columns and 'Time' in clean_df.columns and
        'Booking Date' in clean_df.columns and 'Arrest Date' in clean_df.columns):

        # Count missing values
        booking_time_na_count = clean_df['Booking Time'].isna().sum()
        booking_date_na_count = clean_df['Booking Date'].isna().sum()
        print(f"Fixing {booking_time_na_count} missing 'Booking Time' and {booking_date_na_count} missing 'Booking Date' values")

        # Calculate typical time difference between arrest and booking for existing pairs
        time_diffs = []
        day_changes = 0
        valid_times = clean_df.dropna(subset=['Time', 'Booking Time'])

        for _, row in valid_times.iterrows():
            arrest_time = float(row['Time'])
            booking_time = float(row['Booking Time'])

            # Convert to minutes for easier calculation
            arrest_minutes = (int(arrest_time / 100) * 60) + (arrest_time % 100)
            booking_minutes = (int(booking_time / 100) * 60) + (booking_time % 100)

            # Calculate difference, accounting for day crossing
            if booking_minutes < arrest_minutes:
                # Booking is on the next day
                diff_minutes = (24 * 60 - arrest_minutes) + booking_minutes
                day_changes += 1
            else:
                diff_minutes = booking_minutes - arrest_minutes

            time_diffs.append(diff_minutes)

        # Calculate the median time difference (more robust than mean)
        if time_diffs:
            median_diff_minutes = np.median(time_diffs)
            print(f"Median time between arrest and booking: {median_diff_minutes:.1f} minutes")
            print(f"Detected {day_changes} of {len(valid_times)} cases ({day_changes/len(valid_times)*100:.1f}%) where booking crosses to next day")
        else:
            # Default to 2 hours if we can't calculate
            median_diff_minutes = 120
            print("Using default booking delay of 120 minutes")

        # First, fill any missing booking dates with arrest date (will be adjusted later if needed)
        if booking_date_na_count > 0:
            clean_df['Booking Date'] = clean_df['Booking Date'].fillna(clean_df['Arrest Date'])
            print(f"Initially filled {booking_date_na_count} missing 'Booking Date' values with 'Arrest Date'")

        # Process rows with missing booking times
        date_adjustments = 0
        for idx in clean_df[clean_df['Booking Time'].isna()].index:
            arrest_time = clean_df.loc[idx, 'Time']

            if pd.notna(arrest_time):
                try:
                    # Convert to float if it's not already
                    arrest_time_float = float(arrest_time)

                    # Convert to total minutes
                    arrest_hours = int(arrest_time_float / 100)
                    arrest_minutes = int(arrest_time_float % 100)
                    total_arrest_minutes = arrest_hours * 60 + arrest_minutes

                    # Add median difference
                    total_booking_minutes = total_arrest_minutes + median_diff_minutes

                    # Check if booking crosses to next day
                    crosses_midnight = total_booking_minutes >= 24 * 60

                    # Handle day overflow
                    if crosses_midnight:
                        total_booking_minutes -= 24 * 60

                        # Also adjust the booking date if it crosses midnight
                        if pd.notna(clean_df.loc[idx, 'Booking Date']) and pd.notna(clean_df.loc[idx, 'Arrest Date']):
                            # Convert dates to datetime objects
                            try:
                                arrest_date = pd.to_datetime(clean_df.loc[idx, 'Arrest Date'])
                                booking_date = arrest_date + pd.Timedelta(days=1)
                                clean_df.loc[idx, 'Booking Date'] = booking_date.strftime('%Y-%m-%d')
                                date_adjustments += 1
                            except Exception as e:
                                print(f"Error adjusting date: {e}")

                    # Convert back to military time float
                    booking_hours = int(total_booking_minutes / 60)
                    booking_minutes = int(total_booking_minutes % 60)
                    booking_time = float(booking_hours * 100 + booking_minutes)

                    clean_df.loc[idx, 'Booking Time'] = booking_time

                except (ValueError, TypeError) as e:
                    print(f"Error processing time '{arrest_time}': {e}")
                    continue

        # Check for consistency in existing data - if booking time is earlier than arrest time but dates are the same
        # This indicates a likely error in booking date
        consistency_fixes = 0
        for idx in clean_df.index:
            arrest_time = clean_df.loc[idx, 'Time']
            booking_time = clean_df.loc[idx, 'Booking Time']
            arrest_date = clean_df.loc[idx, 'Arrest Date']
            booking_date = clean_df.loc[idx, 'Booking Date']

            if (pd.notna(arrest_time) and pd.notna(booking_time) and
                pd.notna(arrest_date) and pd.notna(booking_date) and
                arrest_date == booking_date):  # If dates are the same

                # Check if booking time is earlier than arrest time (indicates day crossing)
                if float(booking_time) < float(arrest_time):
                    try:
                        # Booking should be on the next day
                        arrest_date_dt = pd.to_datetime(arrest_date)
                        booking_date_new = arrest_date_dt + pd.Timedelta(days=1)
                        clean_df.loc[idx, 'Booking Date'] = booking_date_new.strftime('%Y-%m-%d')
                        consistency_fixes += 1
                    except Exception as e:
                        print(f"Error fixing date consistency: {e}")

        if consistency_fixes > 0:
            print(f"Fixed {consistency_fixes} cases where booking time indicated next day but booking date was same as arrest date")

        # Check how many were filled
        time_filled_count = booking_time_na_count - clean_df['Booking Time'].isna().sum()
        print(f"Successfully filled {time_filled_count} of {booking_time_na_count} missing 'Booking Time' values")
        print(f"Adjusted {date_adjustments} booking dates to account for overnight bookings")

    return clean_df

def clean_la_crime_data_military_time(df):
    """
    Comprehensive data cleaning with fix for military time format

    Parameters:
    -----------
    df : pandas.DataFrame
        Original crime data DataFrame with military time as floats

    Returns:
    --------
    pandas.DataFrame
        Fully cleaned DataFrame
    """
    print("Starting data cleaning process with military time format fix...")
    print(f"Original dataset shape: {df.shape}")

    # Create a copy of the dataframe to avoid modifying the original
    clean_df = df.copy()

    # Step 1: Drop Cross Street Column
    if 'Cross Street' in clean_df.columns:
        clean_df = clean_df.drop(columns='Cross Street')
        print("Dropped 'Cross Street' column")

    # Step 2: Drop rows with missing values in 'Time' or 'Arrest Type Code'
    rows_before = len(clean_df)
    clean_df = clean_df.dropna(subset=['Time', 'Arrest Type Code'])
    rows_dropped = rows_before - len(clean_df)
    print(f"Dropped {rows_dropped} rows with missing 'Time' or 'Arrest Type Code'")

    # Step 3 & 4: Fix Booking Date and Time together, handling day changes
    clean_df = fix_booking_date_and_time(clean_df)

    # Step 5: Fill Booking Location based on Address, then Area Name as backup
    if 'Booking Location' in clean_df.columns and 'Address' in clean_df.columns:
        booking_loc_na_count = clean_df['Booking Location'].isna().sum()

        # Create mapping from Address to most common Booking Location
        address_to_booking = {}
        valid_bookings = clean_df.dropna(subset=['Booking Location', 'Address'])

        for address, group in valid_bookings.groupby('Address'):
            most_common_location = group['Booking Location'].mode().iloc[0]
            address_to_booking[address] = most_common_location

        # Fill missing booking locations using Address
        for idx in clean_df[clean_df['Booking Location'].isna()].index:
            address = clean_df.loc[idx, 'Address']
            if pd.notnull(address) and address in address_to_booking:
                clean_df.loc[idx, 'Booking Location'] = address_to_booking[address]

        # Check how many were filled by Address
        filled_by_address = booking_loc_na_count - clean_df['Booking Location'].isna().sum()
        print(f"Filled {filled_by_address} missing 'Booking Location' values based on 'Address'")

        # For remaining missing values, use Area Name as backup
        if 'Area Name' in clean_df.columns and clean_df['Booking Location'].isna().any():
            remaining_before_area = clean_df['Booking Location'].isna().sum()

            # Create mapping from Area Name to most common Booking Location
            area_to_booking = {}
            area_bookings = clean_df.dropna(subset=['Booking Location', 'Area Name'])

            for area, group in area_bookings.groupby('Area Name'):
                most_common_location = group['Booking Location'].mode().iloc[0]
                area_to_booking[area] = most_common_location

            # Fill missing booking locations using Area Name
            for idx in clean_df[clean_df['Booking Location'].isna()].index:
                area = clean_df.loc[idx, 'Area Name']
                if pd.notnull(area) and area in area_to_booking:
                    clean_df.loc[idx, 'Booking Location'] = area_to_booking[area]

            # Check how many were filled by Area Name
            filled_by_area = remaining_before_area - clean_df['Booking Location'].isna().sum()
            print(f"Filled {filled_by_area} additional missing 'Booking Location' values based on 'Area Name'")

    # Step 6: Fill Booking Location Code based on Booking Location
    if 'Booking Location Code' in clean_df.columns and 'Booking Location' in clean_df.columns:
        booking_code_na_count = clean_df['Booking Location Code'].isna().sum()

        # Create mapping from Booking Location to Booking Location Code
        location_to_code = {}
        valid_codes = clean_df.dropna(subset=['Booking Location', 'Booking Location Code'])

        for location, group in valid_codes.groupby('Booking Location'):
            most_common_code = group['Booking Location Code'].mode().iloc[0]
            location_to_code[location] = most_common_code

        # Fill missing booking location codes
        for idx in clean_df[clean_df['Booking Location Code'].isna()].index:
            location = clean_df.loc[idx, 'Booking Location']
            if pd.notnull(location) and location in location_to_code:
                clean_df.loc[idx, 'Booking Location Code'] = location_to_code[location]

        # Check how many were filled
        filled_count = booking_code_na_count - clean_df['Booking Location Code'].isna().sum()
        print(f"Filled {filled_count} missing 'Booking Location Code' values based on 'Booking Location'")

    # Step 7: Fill charge-related columns - first using frequent charges
    charge_columns = ['Charge Group Code', 'Charge Group Description', 'Charge Description', 'Disposition Description']

    # Filter to focus on charges that appear at least 100 times
    charge_counts = clean_df['Charge'].value_counts()
    frequent_charges = charge_counts[charge_counts >= 100].index

    print(f"Using {len(frequent_charges)} frequent charges (appearing ≥100 times) out of {len(charge_counts)} total charges")

    # Create mappings for each charge-related column
    charge_mappings = {}

    for column in charge_columns:
        if column in clean_df.columns:
            # Create mapping from 'Charge' to this column
            mapping = {}
            valid_data = clean_df[clean_df['Charge'].isin(frequent_charges)].dropna(subset=['Charge', column])

            for charge, group in valid_data.groupby('Charge'):
                most_common_value = group[column].mode().iloc[0]
                mapping[charge] = most_common_value

            charge_mappings[column] = mapping
            print(f"Created mapping from 'Charge' to '{column}' with {len(mapping)} unique mappings")

    # Apply mappings to fill missing values
    for column in charge_columns:
        if column in charge_mappings:
            column_na_count = clean_df[column].isna().sum()

            # Only apply to rows with frequent charges
            for idx in clean_df[(clean_df[column].isna()) & (clean_df['Charge'].isin(frequent_charges))].index:
                charge = clean_df.loc[idx, 'Charge']
                if charge in charge_mappings[column]:
                    clean_df.loc[idx, column] = charge_mappings[column][charge]

            # Check how many were filled
            filled_count = column_na_count - clean_df[column].isna().sum()
            print(f"Filled {filled_count} missing '{column}' values based on 'Charge'")

    # Step 8: Use charge prefixes for less common charges
    print("\nHandling less frequent charges...")

    # Create charge prefixes
    if 'Charge' in clean_df.columns:
        # Extract prefix from charge code - looking for patterns like "PC", "HS", "VC", etc.
        clean_df['Charge_Prefix'] = clean_df['Charge'].str.extract(r'([A-Za-z]{1,3})(?:\d|\(|$)')[0].str.upper()

        for column in charge_columns:
            if column in clean_df.columns and clean_df[column].isna().any():
                before_count = clean_df[column].isna().sum()

                # Create mapping from prefix to most common value
                prefix_to_value = {}
                valid_data = clean_df.dropna(subset=[column, 'Charge_Prefix'])

                for prefix, group in valid_data.groupby('Charge_Prefix'):
                    if len(group) >= 5 and pd.notna(prefix):  # Only use prefixes with sufficient examples
                        most_common = group[column].mode().iloc[0]
                        prefix_to_value[prefix] = most_common

                # Apply mapping
                for idx in clean_df[clean_df[column].isna()].index:
                    prefix = clean_df.loc[idx, 'Charge_Prefix']
                    if pd.notna(prefix) and prefix in prefix_to_value:
                        clean_df.loc[idx, column] = prefix_to_value[prefix]

                filled = before_count - clean_df[column].isna().sum()
                print(f"Filled {filled} missing '{column}' values using charge code prefixes")

        # Remove the temporary column
        clean_df = clean_df.drop(columns=['Charge_Prefix'])

    # Step 9: Use relationships between charge columns
    column_relationships = [
        ('Charge Group Code', 'Charge Group Description'),
        ('Charge Group Code', 'Charge Description'),
        ('Charge Group Description', 'Charge Group Code'),
        ('Charge Group Description', 'Charge Description')
    ]

    print("\nUsing relationships between columns...")

    for source_col, target_col in column_relationships:
        if source_col in clean_df.columns and target_col in clean_df.columns:
            if clean_df[target_col].isna().any():
                before_count = clean_df[target_col].isna().sum()

                # Create mapping
                col_to_col = {}
                valid_data = clean_df.dropna(subset=[source_col, target_col])

                for val, group in valid_data.groupby(source_col):
                    most_common = group[target_col].mode().iloc[0]
                    col_to_col[val] = most_common

                # Apply mapping
                for idx in clean_df[(clean_df[target_col].isna()) & (clean_df[source_col].notna())].index:
                    val = clean_df.loc[idx, source_col]
                    if val in col_to_col:
                        clean_df.loc[idx, target_col] = col_to_col[val]

                filled = before_count - clean_df[target_col].isna().sum()
                if filled > 0:
                    print(f"Filled {filled} missing '{target_col}' values using '{source_col}'")


    # Final count of missing values
    print("\nRemaining missing values after cleaning:")
    print(clean_df.isna().sum())

    return clean_df

clean_df = clean_la_crime_data_military_time(df)

# Final step: Drop remaining rows with missing values
before_drop = len(clean_df)
clean_df = clean_df.dropna(subset=['Charge Group Code', 'Charge Group Description', 'Charge Description', 'Disposition Description'])
after_drop = len(clean_df)
print(f"Dropped {before_drop - after_drop} rows with remaining missing values")
print(f"\nFinal dataset shape: {clean_df.shape}")

# Extra

# Filtered to only look at adults (people over the age of 18)
# About 4k rows had LAT and LON values of 0, so we removed those rows as well.
clean_df = clean_df[clean_df['Age'] >= 18]
clean_df = clean_df[(clean_df['LAT'] != 0) & (clean_df['LON'] != 0)]

Starting data cleaning process with military time format fix...
Original dataset shape: (334577, 25)
Dropped 'Cross Street' column
Dropped 22 rows with missing 'Time' or 'Arrest Type Code'
Fixing 75378 missing 'Booking Time' and 75378 missing 'Booking Date' values
Median time between arrest and booking: 157.0 minutes
Detected 41699 of 259177 cases (16.1%) where booking crosses to next day
Initially filled 75378 missing 'Booking Date' values with 'Arrest Date'
Fixed 24 cases where booking time indicated next day but booking date was same as arrest date
Successfully filled 75378 of 75378 missing 'Booking Time' values
Adjusted 6576 booking dates to account for overnight bookings
Filled 72932 missing 'Booking Location' values based on 'Address'
Filled 2487 additional missing 'Booking Location' values based on 'Area Name'
Filled 75382 missing 'Booking Location Code' values based on 'Booking Location'
Using 274 frequent charges (appearing ≥100 times) out of 4918 total charges
Created mapping